# Testing Point-in-Polygon methods

In [ ]:
import numpy as np

# regular polygon for testing
lenpoly = 100
polygon = [[np.sin(x)+1,np.cos(x)+1] for x in np.linspace(0,2*np.pi,lenpoly)[:-1]]

# random points set of points to test 
N = 10000
points = 2*np.random.rand(N,2)

### Shapely

In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

spolygon = Polygon(polygon)
%time a = [spolygon.contains(Point(point)) for point in points]

### Ray Tracing

In [ ]:
# Ray tracing
def ray_tracing(x,y,poly):

    n = len(poly)
    inside = False

    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside

%time inside = [ray_tracing(point[0], point[1], polygon) for point in points]

### Ray Tracing with Jit

In [ ]:
from numba import jit 

@jit(nopython=True)
def ray_tracing_jit(x,y,poly):
    n = len(poly)
    inside = False
    p2x = 0.0
    p2y = 0.0
    xints = 0.0
    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside

polygon2 = np.array(polygon)
# first time jit gets compiled, so run twice, time second time
inside = [ray_tracing_jit(point[0], point[1], polygon2) for point in points]
%time inside = [ray_tracing_jit(point[0], point[1], polygon2) for point in points]

### Matplotlib 

In [ ]:
import matplotlib.path as mpltPath
path = mpltPath.Path(polygon)
%time inside = path.contains_points(points)

### Plot Results 

In [ ]:
import matplotlib.pyplot as plt

points_in = points[inside]
x,y = zip(*polygon)
xp, yp = zip(*points)
xp2, yp2 = zip(*points_in)

plt.plot(x,y,'k')
plt.plot(xp,yp,'b.')
plt.plot(xp2,yp2,'g.')